### Pipeline test ###

In this notebook we will try a possible textual claim generation to be implemented later as part of the package

In [1]:
from hydra.core.global_hydra import GlobalHydra
from hydra import initialize,compose
from src.evidence import FeverousRetrieverRandom
from src.pipeline import ClaimGeneratorPipeline
from src.claim import FeverousGenerator
from src.claim import ToTToGenerator
from src.claim import TextualClaim
import numpy as np
import tensorflow_text
import json
import tensorflow as tf

tf.get_logger().setLevel('ERROR')
GlobalHydra.instance().clear()
initialize(config_path='../src/config/', job_name="test_app")

Since the GPL-licensed package `unidecode` is not installed, using Python's `unicodedata` package which yields worse results.
2022-05-17 15:22:12.848696: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-17 15:22:12.848720: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


hydra.initialize()

In [2]:
cfg = compose(config_name="config_pipeline")
rng = np.random.default_rng(cfg.seed)
retrievers = [FeverousRetrieverRandom(p_dataset='../datasets/filtereddb_st_2.db',
                                    num_positive=1,
                                    num_negative=1,
                                    wrong_cell=2,
                                    table_per_page=cfg.table_per_page,
                                    evidence_per_table=cfg.evidence_per_table,
                                    column_per_table=cfg.column_per_table,
                                    seed=23,
                                    key_strategy=strat
                                    ) for strat in ['entity','random']]


generator1 = FeverousGenerator(encoding='compact', model_path='../models/t5_train_feverous.bin')
#generator2 = ToTToGenerator(encoding='totto', model_path='../models/exported_totto_large/1648208035')
generator3 = ToTToGenerator(encoding='compact', model_path='../models/exported_totto_large/1648208035', verbose=True)

generators = [generator3]

pipeline = ClaimGeneratorPipeline([retrievers,generators])
claims = pipeline.generate() # Right now, FeverousRetriever doesn't support an input table


json_evidence = TextualClaim.to_json(claims)
with open('data.json', 'w', encoding='utf-8') as f:
    json.dump(json_evidence, f, ensure_ascii=False, indent=4)



2022-05-17 15:22:17.162642: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-17 15:22:17.163128: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-17 15:22:17.163169: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-05-17 15:22:17.163205: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-05-17 15:22:17.164642: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co